# Entrega 2:

Marvin Daniel Cely Báez

Laura Camila Rojas Valbuena

# Problema 1

Algoritmo de Dijkstra

In [2]:
#Tomado del Algoritmo de Andres jaque
from heapq import heappush, heappop

def updateheap(heap,d,v):
    for i in range(len(heap)):
        if heap[i][1] == v:
            heap[i][0] = d
            fix_minheap(heap,i) # heap=heapify(heap) ineficient      
            break    

def fix_minheap(heap, i):
    if i == 0: return  
    p = int(i/2) #parent  
    if p >= 0 and heap[p][0] > heap[i][0]:
        heap[i], heap[p] = heap[p], heap[i]
        fix_minheap(heap,p)   
            
def Dijkstra(G,start):
        
    D = {} # dictionary of final distances
    for v in G:
        D[v] = float('inf')
    D[start] = 0
    
    P = {} # dictionary of predecessors
    
    Q=[] #  priority queue est.dist. of non-final vert.
    for v in G:
        item = []
        item.append(D[v])
        item.append(v)
        heappush(Q,item)
    
    
    #S = []
    while Q:
        u = heappop(Q)[1]
        #S.append(u)
        for v in G[u]:
            newDuv = D[u] + G[u][v]
            if newDuv < D[v]:
                P[v] = u
                D[v] = newDuv
                updateheap(Q,D[v],v)
    return D,P

In [3]:
#Tomado del Algoritmo de Andres jaque
def shortestPath(G,start,end,method):
    if method ==  'Dijkstra':
        D,P = Dijkstra(G,start)
    elif method == 'Bellman':
        D,P = BellmanFord(G,start)
    else:
        D,P = Dijkstra(G,start)
    
    Path = []
    while 1:
        Path.append(end)
        if end == start: break
        end = P[end]
    Path.reverse()
    return Path

Calcular Promedios entre sus distancias por cada vértice

In [4]:
def calcularDistancia(G,path):
    distancia = 0.0
    for i in range(1,len(path)):
        distancia = distancia + G[path[i-1]][path[i]]
    return distancia
#Se crea una matriz de adjacencia, su contenido son la distancia entre 2 vertices
def sumVertice(G,v,mask):
    distancias = {}
    Vertice = G.keys()
    for i in range(len(Vertice)):
        if Vertice[i] not in mask:
            distancias[Vertice[i]] = calcularDistancia(G,shortestPath(G,v,Vertice[i],'Dijkstra'))
    return distancias

def ReflejarMatriz(Dic):
    for key, value in Dic.iteritems(): 
        for key1, cost in value.iteritems():
            if key != key1:
                Dic[key1][key] = cost
    return Dic
                
def MatrizPathBarrios(G):    
    Vertice = G.keys()
    Vertice.sort()
    SumVertice = {}
    mask = []
    for i in range(len(Vertice)):
        mask.append(Vertice[i])
        SumVertice[Vertice[i]] = sumVertice(G,Vertice[i],mask)
    return ReflejarMatriz(SumVertice)

def PromPathbarrios(G):
    prom = {}
    for key, value in G.iteritems():
        suma = 0.0
        for key1, cost in value.iteritems():
            suma = suma + cost
        prom[key] = suma/len(value)    
    return prom

In [5]:
def ApartarTienda(G,tienda):
    del G[tienda]
    for key,value in G.iteritems():
        del value[tienda]
        
def haySimil(s,avgMinCost):     
    parecido = []
    for key, value in avgMinCost.iteritems():
        if value == s:
            parecido.append(key)
    return parecido

def promCaminos(G1,v):
    sumCost = 0.0
    vecinos = G1[v]    
    for key, cost in vecinos.iteritems():
        sumCost = sumCost + cost
    return sumCost/len(vecinos)


def ElegirMejorTienda(G1,simil):
    promVecinos = []
    for i in range(len(simil)):
        promVecinos.append(promCaminos(G1,simil[i]))
    #Elegir prom mas grande
    maxi = min(promVecinos)#CAMBIIEEEEE ESTOOOOOOO A MENOR------------------------  
    elElegido = 'inf'
    for i in range(len(simil)):
        if maxi == promVecinos[i]:
            elElegido = simil[i]
    return elElegido  

def ubicarTiendas(G,numTiendas):
    tiendasElegidas = []
    G1 = G.copy()
    if numTiendas == len(G):
        return G.keys()
    #Las tiendas tienen que ser menores a la cantidad de Barrios
    elif numTiendas < len(G):
        for i in range(numTiendas):            
            #Elegir el promedio pas pequeño
            avgMinCost = PromPathbarrios(MatrizPathBarrios(G))   
            menor = 99.0
            tienda = 'inf'
            for key,value in avgMinCost.iteritems():
                if value < menor:                                       
                    tienda = key
                    menor = value
            #Elegir la mejor tienda
            simil = haySimil(menor,avgMinCost)
            if len(simil) > 1:
                tienda = ElegirMejorTienda(G1,simil)
            tiendasElegidas.append(tienda)
            ApartarTienda(G,tienda)#Las tiendas elegidas no se tendran en cuenta para la siguiente iteración
        return tiendasElegidas
    else:
        print "Santos frijoles saltarines!!! el número de tiendas no esta entre las cantidades de barrios :'("

In [13]:
def ElegirMejorTiendaCosto(G1,simil):
    promVecinos = []
    for i in range(len(simil)):
        promVecinos.append(promCaminos(G1,simil[i]))
    #Elegir prom mas grande
    maxi = max(promVecinos)#CAMBIIEEEEE ESTOOOOOOO A MENOR------------------------  
    elElegido = 'inf'
    for i in range(len(simil)):
        if maxi == promVecinos[i]:
            elElegido = simil[i]
    return elElegido  

def addCostBuilt(avgMinCost, C):
    v = avgMinCost.keys()
    for i in range(len(v)):
        avgMinCost[v[i]] = avgMinCost.get(v[i])/C[v[i]]

def ubicarTiendasCostos(G,C,numTiendas):
    tiendasElegidas = []
    G1 = G.copy()
    if numTiendas == len(G):
        return G.keys()
    #Las tiendas tienen que ser menores a la cantidad de Barrios
    elif numTiendas < len(G):
        for i in range(numTiendas):            
            #Elegir el promedio mas pequeño
            avgMinCost = PromPathbarrios(MatrizPathBarrios(G))   
            menor = 99.0
            tienda = 'inf'
            addCostBuilt(avgMinCost,C)
            for key,value in avgMinCost.iteritems():
                if value < menor:                                       
                    tienda = key
                    menor = value
            #Elegir la mejor tienda
            simil = haySimil(menor,avgMinCost)
            if len(simil) > 1:
                tienda = ElegirMejorTiendaCosto(G1,simil)
            tiendasElegidas.append(tienda)
            ApartarTienda(G,tienda)#Las tiendas elegidas no se tendran en cuenta para la siguiente iteración
        return tiendasElegidas
    else:
        print "Santos frijoles saltarines!!! el número de tiendas no esta entre las cantidades de barrios :'("

Costo de la solucion

In [14]:
def CostoSolucion(G, sol):
    G1 = MatrizPathBarrios(G)
    costo = []
    for key, value in G1.iteritems():
        seleccionados = []
        for i in range(len(sol)):
            seleccionados.append(value.get(sol[i],0))
        costo.append(min(seleccionados))
    return sum(costo)

def CostoSolucionBuilt(G, C, sol):
    G1 = MatrizPathBarrios(G)
    costo = []
    built = 0
    for key, value in G1.iteritems():
        seleccionados = []
        for i in range(len(sol)):
            seleccionados.append(value.get(sol[i],0))
        costo.append(min(seleccionados))
    for i in range(len(sol)):
        built += C[sol[i]]
    return sum(costo)+built

In [15]:
def imprimirBarrios(solucion):
    s = ''
    for i in range(len(solucion)):
        s = s + solucion[i]+','
    return s

Lectura de Archivo

In [16]:
def lecturaGrafo(arch):
    infile = open(arch, 'r')
    numBarrios = int(infile.readline())
    G = {}
    for line in infile:
        vector = []
        vector = line.split()
        key = vector[0]
        subG = {}
        for i in range(1,len(vector)):
            if int(vector[i]) != 0:
                key1 = chr(64+i)
                cost = vector[i]
                subG[key1] = int(cost)
        G[key] = subG
    infile.close()
    return numBarrios, G

def lecturaGrafoCosto(arch):
    infile = open(arch, 'r')
    numBarrios = int(infile.readline())
    G = {}
    C = {}
    for line in infile:
        vector = []
        vector = line.split()
        key = vector[0]
        built = int(vector[1])
        subG = {}
        for i in range(2,len(vector)):
            if int(vector[i]) != 0:
                key1 = chr(63+i)
                cost = vector[i]
                subG[key1] = int(cost)
        G[key] = subG
        C[key] = built
    infile.close()
    return numBarrios, G, C

In [21]:
def Punto1_1(nombreArchivo):
    numBarrios,G = lecturaGrafo(nombreArchivo)
    numBarrios,H = lecturaGrafo(nombreArchivo)
    sol = ubicarTiendas(G,numBarrios)
    print int(CostoSolucion(H,sol))
    print imprimirBarrios(sol)
    
def Punto1_2(nombreArchivo):
    numBarrios,G,C = lecturaGrafoCosto(nombreArchivo)
    numBarrios,H,C = lecturaGrafoCosto(nombreArchivo)
    sol = ubicarTiendasCostos(G,C,numBarrios)
    print int(CostoSolucionBuilt(H,C,sol))
    print imprimirBarrios(sol)
    
    print G

Punto 1 Parte 1

In [22]:
print 'input_n5_k1' 
Punto1_1('input_n5_k1')
print ''
print 'input_n5_k2' 
Punto1_1('input_n5_k2')
print ''
print 'input_n5_k3' 
Punto1_1('input_n5_k3')
print ''


input_n5_k1
23
A,

input_n5_k2
8
A,C,

input_n5_k3
6
A,C,D,



Punto 1 parte 2 (Costos de Construccion)

In [23]:
#Mejorar el resultado...Optimizarlo...Corregirlo
print 'input_n5_k1_c' 
Punto1_2('input_n5_k1_c')
print ''
print 'input_n5_k2_c' 
Punto1_2('input_n5_k2_c')
print ''
print 'input_n5_k3_c' 
Punto1_2('input_n5_k3_c')
print ''

input_n5_k1_c
58
A,
{'C': {'B': 9, 'D': 18}, 'B': {'C': 9, 'D': 19}, 'D': {'C': 18, 'B': 19}}

input_n5_k2_c
63
A,C,
{'B': {'D': 19}, 'D': {'B': 19}}

input_n5_k3_c
73
A,C,B,
{'D': {}}



Fin :)